In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

# SNGP new exps

## CoNLL-2003

In [37]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [38]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py seeds=[23419,705525,4837,10671619,1084218,43]'
                subsample_perc = conf['subsample_perc']
                args = ' '.join(generate_func[i](conf))
                
                with open(f'../../workdir/hp_search/electra_raw_sngp/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'                
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/electra_raw_sngp/{task_name}/{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../scripts/ner_scripts/run_train_models_sngp_ner_new.sh', 'w') as rsh:
        rsh.write(full_config)

In [39]:
tasks = ['conll2003']

train_configs = {
    'use_selective': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

In [40]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    config_list.append('training\=electra_base')
    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_conll2003.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_raw_sngp/{task_name}/{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_sngp/{task_name}/sngp'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [41]:
tasks = ['conll2003']

eval_configs = {
    'ue': ['sngp'],
    'use_selective': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
    'per_device_eval_batch_size': [128],
    'committee_size': [10]
}
    
cuda_devices = '[0,1]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp_ner_new.sh')

In [42]:
full_test = 'bash run_train_models_sngp_ner_new.sh\n'
full_test += 'bash run_eval_sngp_ner_new.sh'

with open (f'../../scripts/ner_scripts/run_electra_sngp_ner_new.sh', 'w') as rsh:
    rsh.write(full_test)

## GLUE

In [27]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    
    config_list.append('training\=electra_base')
    return config_list

In [30]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_glue.py seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                with open(f'../../workdir/hp_search/electra_raw_sngp/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'                
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                new_task+=f' output_dir=../workdir/run_train_models/electra_raw_sngp/{task_name}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../scripts/miscl_scripts/run_train_models_sngp_new.sh', 'w') as rsh:
        rsh.write(full_config)

In [31]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'use_selective': ['False'],
}
    
cuda_devices = '[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

In [34]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    config_list.append('training\=electra_base')
    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_glue.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_raw_sngp/{task_name}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_sngp/{task_name}/sngp'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [35]:
tasks = ['mrpc', 'cola', 'sst2']

eval_configs = {
    'ue': ['sngp'],
    'use_selective': ['False'],
    'per_device_eval_batch_size': [128],
    'committee_size': [10]
}
    
cuda_devices = '[0,1]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp_new.sh')

In [36]:
full_test = 'bash run_train_models_sngp_new.sh\n'
full_test += 'bash run_eval_sngp_new.sh'

with open (f'../../scripts/miscl_scripts/run_electra_sngp_new.sh', 'w') as rsh:
    rsh.write(full_test)

# SNGP training HP search

In [3]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('training\=electra_base')
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_glue_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']

                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_raw_sngp/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'reg_type': ['raw'],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_sngp.sh')

In [8]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_conll2003_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']

                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_raw_sngp/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/ner_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [9]:
tasks = ['conll2003']

train_configs = {
    'reg_type': ['raw'],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_sngp.sh')

# OOD with noise train

In [58]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    #config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [59]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'
                    noise_perc = conf['noise_perc']

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [60]:
tasks = ['clinc', 'rostd']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],#['raw', 'metric', 'reg-curr'], 
    'use_spectralnorm': ['False'],# 'True'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_noise.sh')

In [61]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ue.margin\={margin}'
                    args += f' ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=64'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    noise_perc = conf['noise_perc']

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [63]:
tasks = ['clinc', 'rostd']

ue_configs = {
    'ue': ['nuq', 'mahalanobis', 'l_nuq', 'l_mahalanobis'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_noise.sh')

In [54]:
full_test = 'bash run_train_electra_models_noise.sh\n'
full_test += 'bash run_eval_electra_noise.sh'

with open (f'../../scripts/ood_scripts/run_exps_noise.sh', 'w') as rsh:
    rsh.write(full_test)

# Run exps for model with SN for residuals

In [41]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('spectralnorm_layer\={}'.format(config['spectralnorm_layer']))
    
    config_list.append('training\=electra_base')
    return config_list

In [60]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}_ood.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                spectralnorm_layer = conf['spectralnorm_layer']
                if 'resid' in spectralnorm_layer and use_spectralnorm == 'sn':
                    use_spectralnorm+='_all'
                    
                path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'

                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'

                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/'

                full_config += '\n' + new_task if len(full_config) else new_task

    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [62]:
tasks = ['20newsgroups', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'], 
    'use_spectralnorm': ['True'],
    'script': ['run_ood.py'],
    'spectralnorm_layer': ['resid+last']
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_sn_all.sh')

# Eval determenistic methods

In [68]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('spectralnorm_layer\={}'.format(config['spectralnorm_layer']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}_ood.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'

                args = ' '.join(generate_func[i](conf))

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                spectralnorm_layer = conf['spectralnorm_layer']
                if 'resid' in spectralnorm_layer and use_spectralnorm == 'sn':
                    use_spectralnorm+='_all'

                path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'

                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'

                args += f' training.per_device_eval_batch_size\=256'

                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                ood_data = conf['ood_data']
                if task_name == ood_data:
                    continue
                elif ood_data in task_name:
                    continue

                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{ood_data}'

                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path

                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/models/{task_name}_ood'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
 
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [69]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['l_nuq', 'l_mahalanobis', 'decomposing_md', 'nuq', 'mahalanobis', 'ddu'],
    'use_spectralnorm': ['True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'newsgroup', 'rte', 'sst2'],
    'spectralnorm_layer': ['resid+last']
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_sn_all.sh')

In [70]:
full_test = 'bash run_train_electra_models_new_benchmark_sn_all.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_sn_all.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_sn_all.sh', 'w') as rsh:
    rsh.write(full_test)

# HP Search for SN for Resid

In [3]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('spectralnorm_layer\={}'.format(config['spectralnorm_layer']))
    
    config_list.append('training\=electra_base')
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}_ood.yaml python ./run_ood_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                if reg == 'metric' and task_name=='clinc':
                    continue
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}_all/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['sst2', '20newsgroups']

train_configs = {
    'reg_type': ['reg-curr', 'metric', 'raw'],
    'use_spectralnorm': ['True'],
    'spectralnorm_layer': ['resid+last']
}
    
cuda_devices = [0, 1, 2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_all_sn.sh')

# OOD exps with new benchmark

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    subsample_perc = conf['subsample_perc']
                    
                    if task_name == '20newsgroups':
                        path = f'../../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{use_spectralnorm}.yaml'
                    else:
                        path = f'../../../hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [5]:
tasks = ['20newsgroups', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'], 
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_subsamples.sh')

In [2]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    subsample_perc = conf['subsample_perc']

                    if task_name == '20newsgroups':
                        path = f'../../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{use_spectralnorm}.yaml'
                    else:
                        path = f'../../../hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ue.margin\={margin}'
                    args += f' ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=256'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    ood_data = conf['ood_data']
                    if task_name == ood_data:
                        continue
                    elif ood_data in task_name:
                        continue

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}/{ood_data}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}/models/{task_name}_ood'
                    new_task+=f' model_series_dir={model_series_dir}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [5]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],#['l_nuq', 'l_mahalanobis', 'decomposing_md', 'nuq', 'mahalanobis', 'ddu', 'sngp'],
    'use_spectralnorm': ['False'],# 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'newsgroup', 'rte', 'sst2'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3],#[0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_subsamples.sh')

In [55]:
full_test = 'bash run_train_electra_models_new_benchmark_subsamples.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_subsamples.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

# OOD exps with determenistic methods

In [4]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    #config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [5]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'
                    subsample_perc = conf['subsample_perc']

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [22]:
tasks = ['clinc', 'rostd']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],#['raw', 'metric', 'reg-curr'], 
    'use_spectralnorm': ['False'],# 'True'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_subsamples.sh')

In [6]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    #config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ue.margin\={margin}'
                    args += f' ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=64'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    subsample_perc = conf['subsample_perc']

                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{subsample_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['clinc', 'rostd']

ue_configs = {
    #'ue': ['decomposing_md', 'nuq', 'mahalanobis', 'ddu', 'sngp'],
    'ue': ['l_nuq', 'ddu', 'nuq', 'mahalanobis', 'l_mahalanobis'],
    'use_spectralnorm': ['False'],# 'True'],
    'reg_type': ['raw'],# 'metric', 'reg-curr'],
    'script': ['run_ood.py'],
    #'validation_subsample': [0.0],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_subsamples.sh')

In [8]:
full_test = 'bash run_train_electra_models.sh\n'
full_test += 'bash run_eval_electra.sh'

with open (f'../../scripts/ood_scripts/run_exps.sh', 'w') as rsh:
    rsh.write(full_test)

In [9]:
full_test = 'bash run_train_electra_models_subsamples.sh\n'
full_test += 'bash run_eval_electra_subsamples.sh'

with open (f'../../scripts/ood_scripts/run_exps_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

# OOD datasets HP Search

In [4]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_ood_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                if reg == 'metric' and task_name=='clinc':
                    continue
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['clinc', 'rostd']#, 'snips']

train_configs = {
    'reg_type': ['reg-curr', 'metric'],#['raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_regs.sh')

# Decomposing for GLUE misclassification

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    #config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
           
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [8]:
tasks = ['mrpc', 'cola', 'sst2', 'sst5']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[1,4,7]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models.sh')

In [4]:
tasks = ['sst5']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],#['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False'],# 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst5.sh')

In [12]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if (config['ue'] == 'mc_mahalanobis') or (config['ue'] == 'mc'):
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    #config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ++ue.margin\={margin}'
                args += f' ++ue.lamb_intra\={lamb_intra}'
                args += f' ++ue.lamb\={lamb}'
                
                args += f' training.per_device_eval_batch_size\=64'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [11]:
tasks = ['mrpc', 'cola', 'sst2', 'sst5']

ue_configs = {
    'ue': ['decomposing_md', 'nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    #'subsample_perc': [0.1]
}
    
cuda_devices = '[1,4,7]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_decomposing.sh')

In [13]:
tasks = ['sst5']

ue_configs = {
    'ue': ['mahalanobis', 'mc'],
    'use_spectralnorm': ['False'],# 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[0,1]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_sst5.sh')

# Twitter HSO NUQ and Mahalanobis

In [12]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [13]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [14]:
tasks = ['twitter_hso']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.0],
}
    
cuda_devices = '[1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_twitter_hso.sh')

In [1]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
                
                args += f' training.per_device_eval_batch_size\=64'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['twitter_hso']

ue_configs = {
    'ue': ['ddu', 'nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.0]
}
    
cuda_devices = '[1,2,7]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_twitter_hso_det_best.sh')

# Twitter HSO HP search

In [75]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [78]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_20newsgroups_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [80]:
tasks = ['twitter_hso']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2, 3]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_twitter_hso.sh')

# MNLI NUQ, Mahalanobis

In [67]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [68]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [69]:
tasks = ['mnli']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_mnli.sh')

In [70]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [71]:
tasks = ['mnli']

ue_configs = {
    'ue': ['ddu', 'nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_mnli_det_best.sh')

# MNLI HP SEARCH

In [59]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [62]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_glue_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [63]:
tasks = ['mnli']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2, 3, 4, 5]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_mnli.sh')

# Train SST-2 with different size of dataset

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [28]:
tasks = ['sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst2_dif_size.sh')

In [4]:
tasks = ['mrpc']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_mrpc_dif_size.sh')

In [11]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=-40')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [12]:
tasks = ['sst2']

ue_configs = {
    'ue': ['l_mahalanobis', 'nuq', 'mahalanobis', 'ddu', 'l_nuq'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05],
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst2_det_best_dif_size.sh')

In [13]:
tasks = ['mrpc']

ue_configs = {
    'ue': ['nuq', 'mahalanobis', 'ddu', 'l_mahalanobis', 'l_nuq'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_mrpc_det_best_dif_size.sh')

In [13]:
full_test = 'bash run_train_electra_models_sst2_dif_size.sh\n'
full_test += 'bash run_train_electra_models_mrpc_dif_size.sh\n'
full_test += 'bash run_eval_electra_models_sst2_det_best_dif_size.sh\n'
full_test += 'bash run_eval_electra_models_mrpc_det_best_dif_size.sh'

with open (f'../../scripts/miscl_scripts/run_exps_diff_size.sh', 'w') as rsh:
    rsh.write(full_test)

# NUQ, Mahalanobis for SST-2

In [26]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [27]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [28]:
tasks = ['sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst2.sh')

In [64]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=-40')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [66]:
tasks = ['sst2']

ue_configs = {
    'ue': ['ddu', 'nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst2_det_best.sh')

# With Noise

In [1]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [2]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [40]:
tasks = ['sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst2_noise.sh')

In [41]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [43]:
tasks = ['sst2']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst2_det_noise.sh')

# Electra train SST5

In [30]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [31]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [32]:
tasks = ['sst5']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.0],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst5.sh')

# Nuq, Mahalanobis

In [11]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')

    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [12]:
tasks = ['sst5']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.0]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst5_det_best.sh')

# Electra HP search for SST5

In [ ]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_20newsgroups_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['sst5']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2, 3]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_sst5.sh')

# Electra train QQP and Amazon with noise

In [23]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [24]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [25]:
tasks = ['qqp']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_qqp_noise.sh')

# NUQ, Mahalanobis with noise

In [20]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=40')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=10')
        config_list.append('+ue.nuq.n_samples\=3')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}/noise_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [22]:
tasks = ['qqp']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_qqp_det_noise.sh')

# Electra train QQP and Amazon

In [ ]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['qqp']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_qqp.sh')

In [ ]:
tasks = ['amazon']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.01]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_amazon.sh')

# NUQ, Mahalanobis

In [16]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=40')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=10')
        config_list.append('+ue.nuq.n_samples\=3')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                #ue_type = conf['ue']
                ue_type = conf['ue'] if conf['ue'] != 'nuq' else 'nuq_best1'
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [17]:
tasks = ['qqp']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_qqp_det_best.sh')

In [ ]:
tasks = ['amazon']

ue_configs = {
    'ue': ['nuq', 'mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': ['run_20newsgroups.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_amazon_det_best.sh')

# MC dropout

In [ ]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))


    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    
    config_list.append('ue.dropout_subs\={}'.format(config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(config['inference_prob']))
    
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                if task_name=='qqp':
                    script = 'run_glue.py'
                else:
                    script = 'run_20newsgroups.py'
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['qqp', 'amazon']

ue_configs = {
    'ue': ['mc'],
    'use_spectralnorm': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'script': [''],
    'validation_subsample': [0.0],
    'subsample_perc': [0.1],
    'calibrate': ['False'],
    'dropout_subs': ['all'],
    'committee_size': [20],
    'inference_prob': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_qqp_amazon_mc_best.sh')

# MC-DPP

In [ ]:
import os 

def generate_eval_dpp_str_config(dpp_config, ds, ddpp):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\=raw')
    config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    ds_to_mf = {'amazon': {'ddpp_dpp': 0.4, 'ddpp_ood': 0.6},
                'qqp': {'ddpp_dpp': 0.6, 'ddpp_ood': 0.5}
    }
    max_frac = ds_to_mf[ds][ddpp]
    config_list.append('ue.dropout.max_frac\={}'.format(max_frac))    
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list

def generate_bash_mcdpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                ddpp = f'ddpp_{dpp_type}'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                if task_name=='qqp':
                    script = 'run_glue.py'
                else:
                    script = 'run_20newsgroups.py'
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf, task_name, ddpp))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                
                with open(f'../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                base_path+= f'/ddpp_{dpp_type}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['qqp', 'amazon']

dpp_configs = {
    'ue': ['mc-dpp'],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mcdpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_qqp_amazon_models_dpp.sh')

# Electra HP search for Amazon

In [ ]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_20newsgroups_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['amazon']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2, 3, 4, 5]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_amazon.sh')

# Electra HP search NEW

In [ ]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_glue_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1, 2]#'[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search.sh')

In [ ]:
tasks = ['qqp']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_qqp.sh')

# DeBERTA HP Search

In [ ]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_glue_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/deberta_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='microsoft/deberta-base'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [0, 1]#'[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'deberta_hp_search.sh')

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)

    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_conll2003_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/deberta_{reg}_{use_spectralnorm}/{task_name}'
                new_task += " model.model_name_or_path='microsoft/deberta-base'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/ner_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['conll2003']

train_configs = {
    'reg_type': ['metric'],#, 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
}
    
cuda_devices = [2]#'[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'deberta_hp_search.sh')

# DeBERTA train all CoNLL-2003

In [34]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    config_list.append('training\=electra_base')
    return config_list

In [35]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                args = ' '.join(generate_func[i](conf))
                                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ue_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [36]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False'],# 'True'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_deberta_models.sh')

# MC-DPP

In [37]:
import os 

def generate_eval_dpp_str_config(dpp_config, ds, ddpp):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    ds_to_mf = {
        'conll2003': {'ddpp_ood': 0.45, 'ddpp_dpp': 0.3}      
    }
    max_frac = ds_to_mf[ds][ddpp]
    config_list.append('ue.dropout.max_frac\={}'.format(max_frac))
    config_list.append('ue.dropout.mask_name_for_mask\=rbf')
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list

def generate_bash_mcdpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                ddpp = f'ddpp_{dpp_type}'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                
                args = ' '.join(generate_func[i](conf, task_name, ddpp))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                base_path+= f'/ddpp_{dpp_type}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'
                
                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/ddpp_{dpp_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/ddpp_{dpp_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [38]:
tasks = ['conll2003']

dpp_configs = {
    'ue': ['mc-dpp'],
    #'max_frac' : [0.3, 0.4, 0.5, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [20],
    'validation_subsample': [0.1],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mcdpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_deberta_models_dpp_best.sh')

# MC dropout

In [39]:
def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.use_cache\=False')
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list


def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                base_path+= f'/mc_all/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'
                
                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/mc_all'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/mc_all extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [40]:
mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'validation_subsample': [0.1],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_deberta_models_mc_best.sh')

# NUQ, Mahalanobis, MC Mahalanobis

In [41]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc_val\={}'.format(config['validation_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [42]:
tasks = ['conll2003']

ue_configs = {
    'ue': ['mahalanobis'],#['nuq', 'mahalanobis', 'mc_mahalanobis'],
    'validation_subsample': [0.1],
    'use_spectralnorm': ['False'],# 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_deberta_models_det_best.sh')

# Deep Ensemble

In [51]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        subsample_perc = 0.1
        args = f'data.subsample_perc\={subsample_perc} ue.calibrate\=True data.subsample_perc_val\=0.1'
        
        with open(f'../../../workdir/hp_search/deberta_raw_no_sn/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        margin = params['margin']
        lamb_intra = params['lamb_intra']
        lamb = params['lamb']
        args += f' +ue.margin\={margin}'
        args += f' +ue.lamb_intra\={lamb_intra}'
        args += f' ue.lamb\={lamb}'
        args += ' model.model_name_or_path\=microsoft/deberta-base'
        
        output_dir = f'../workdir/run_train_ensemble_series/deberta/{task_name}/'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        new_task += f' script=run_conll2003.py'
        
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../scripts/ue_scripts/run_train_deberta_ensemble_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [52]:
tasks = ['conll2003']

cuda_devices = '[0,1,2,3,4,5]'

generate_bash(cuda_devices, tasks)

In [53]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_ensemble_series.yaml python run_tasks_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        
        
        args = f'data.subsample_perc\=0.1 ue.calibrate\=True data.subsample_perc_val\=0.1'
        with open(f'../../../workdir/hp_search/deberta_raw_no_sn/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        margin = params['margin']
        lamb_intra = params['lamb_intra']
        lamb = params['lamb']
        args += f' +ue.margin\={margin}'
        args += f' +ue.lamb_intra\={lamb_intra}'
        args += f' ue.lamb\={lamb}'

        ensemble_series_dir = f'../workdir/run_train_ensemble_series/deberta/{task_name}/ensembles/'
        new_task+=f" args='{args}' ensemble_series_dir={ensemble_series_dir}"
        new_task+=f' output_dir=../workdir/run_tasks_for_ensemble_series/deberta/{task_name}/'
        new_task+=f' script=run_conll2003.py'

        calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_raw_no_sn/{task_name}/ensemble'
        calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_ensemble_series/deberta/{task_name}/final_results extract_config=False output_dir={calc_output_dir};'

        full_config += '\n' + new_task if len(full_config) else new_task
        full_config += '\n' + calc_command

    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [54]:
tasks = ['conll2003']

cuda_devices = '0'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_deberta_ensemble_series_ner.sh')

# DeBERTA train all GLUE

In [43]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    config_list.append('training\=electra_base')
    return config_list

In [44]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                args = ' '.join(generate_func[i](conf))
                
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ue_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [45]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False'],# 'True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_deberta_models_glue.sh')

# MC-DPP

In [46]:
import os 

def generate_eval_dpp_str_config(dpp_config, ds, ddpp):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))

    ds_to_mf = {
        'mrpc': {'ddpp_ood': 0.45, 'ddpp_dpp': 0.6},
        'cola': {'ddpp_ood': 0.45, 'ddpp_dpp': 0.6},
        'sst2': {'ddpp_ood': 0.45, 'ddpp_dpp': 0.6}
    }
    
    max_frac = ds_to_mf[ds][ddpp]
    config_list.append('ue.dropout.max_frac\={}'.format(max_frac)) 
    mask_name_for_mask = 'rbf'
    config_list.append('ue.dropout.mask_name_for_mask\={}'.format(mask_name_for_mask)) 
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list

def generate_bash_mcdpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                ddpp = f'ddpp_{dpp_type}'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf, task_name, ddpp))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                base_path+= f'/ddpp_{dpp_type}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/ddpp_{dpp_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/ddpp_{dpp_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [47]:
tasks = ['mrpc', 'cola', 'sst2']

dpp_configs = {
    'ue': ['mc-dpp'],
    #'max_frac' : [0.3, 0.4, 0.5, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mcdpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_deberta_models_glue_dpp_best.sh')

# MC Dropout

In [48]:
def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list



def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'no_sn'
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                base_path+= f'/mc_all/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/mc_all'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/mc_all extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [49]:
tasks = ['mrpc', 'cola', 'sst2']

mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'validation_subsample': [0.0],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_deberta_models_glue_mc_best.sh')

# Mahalanobis, NUQ, MC Mahalanobis

In [50]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('training.per_device_eval_batch_size\=64') #due to out-of-memory

    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/0.0/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [51]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mahalanobis'],# 'nuq', 'mc_mahalanobis'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False'],# 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_deberta_models_glue_det_best.sh')

# Deep Ensemble

In [55]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        subsample_perc = 0.63 if task_name!='sst2' else 0.1
        args = f'data.subsample_perc\={subsample_perc} ue.calibrate\=True data.validation_subsample\=0.0 do_train\=True do_eval\=False do_ue_estimate\=False'
        
        with open(f'../../../workdir/hp_search/deberta_raw_no_sn/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        args += ' model.model_name_or_path\=microsoft/deberta-base'
        
        output_dir = f'../workdir/run_train_ensemble_series/deberta/{task_name}/'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../scripts/ue_scripts/run_train_deberta_ensemble_models.sh', 'w') as rsh:
        rsh.write(full_config)

In [56]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '[0,1,2,3,4,5]'

generate_bash(cuda_devices, tasks)

In [57]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_ensemble_series.yaml python run_tasks_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'

        ensemble_series_dir = f'../workdir/run_train_ensemble_series/deberta/{task_name}/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir}'
        new_task+=f' output_dir=../workdir/run_tasks_for_ensemble_series/deberta/{task_name}/'

        calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_raw_no_sn/{task_name}/ensemble'
        calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_ensemble_series/deberta/{task_name}/final_results extract_config=False output_dir={calc_output_dir};'

        full_config += '\n' + new_task if len(full_config) else new_task
        full_config += '\n' + calc_command

    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [58]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '0'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_deberta_ensemble_series.sh')

# Deberta MD SN search

In [40]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm_train']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    config_list.append('training\=electra_base')
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    
    return config_list

def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm_inference']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    
    config_list.append('+ue.sn_value\={}'.format(mc_config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(mc_config['eval_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

In [41]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    new_task += f' script=run_glue_dpp_hp.py'

                    args = ' '.join(generate_func[0](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    validation_subsample = conf['validation_subsample']
                    eval_subsample = conf['eval_subsample']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                    
                    with open(f'../../../workdir/hp_search/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    try:
                        margin = params['margin']
                        lamb_intra = params['lamb_intra']
                    except:
                        margin = 0
                        lamb_intra = 0

                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    lamb = params['lamb']
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    sn_value = conf['sn_value']
                    new_task+=f' output_dir=../workdir/run_train_models_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                    full_config += '\n' + new_task if len(full_config) else new_task



                    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    args = ' '.join(generate_func[1](conf))
                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'
                    new_task += f' script=run_glue_dpp_hp.py'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    validation_subsample = conf['validation_subsample']
                    eval_subsample = conf['eval_subsample']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                    sn_value = conf['sn_value']

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/models/{task_name}/'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{sn_value}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command


                    full_config += f'\nrm -rf {model_series_dir}'
                except:
                    pass
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [42]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'calibrate': ['False'],
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
    'sn_value': [0.4, 0.6, 0.8, 1, 2, 3],
    'use_spectralnorm_inference': ['False'],
    'use_spectralnorm_train': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'


generate_bash([ue_configs], cuda_devices, tasks, 
              [generate_train_str_config, generate_ue_eval_methods_str_config], filename='run_train_eval_det_models_with_new_sn_params_deberta.sh')

In [43]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm_train']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['subsample_perc_val']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    config_list.append('ue.use_cache\=False')
    config_list.append('training\=electra_base')
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    
    return config_list

def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm_inference']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=False')
    config_list.append('ue.calibrate\=False')
    
    config_list.append('+ue.sn_value\={}'.format(mc_config['sn_value']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['subsample_perc_val']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    
    return config_list

In [44]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003_dpp_hp.py'

                args = ' '.join(generate_func[0](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                eval_subsample = conf['subsample_perc_val']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                
                with open(f'../../../workdir/hp_search/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                sn_value = conf['sn_value']
                new_task+=f' output_dir=../workdir/run_train_models_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                full_config += '\n' + new_task if len(full_config) else new_task
                
                
                
                base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[1](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                new_task += f' script=run_conll2003_dpp_hp.py'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                eval_subsample = conf['subsample_perc_val']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                sn_value = conf['sn_value']
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/models/{task_name}/'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{sn_value}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series_sn_params/deberta_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                
                
                full_config += f'\nrm -rf {model_series_dir}'
    with open (f'../../scripts/ner_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [45]:
tasks = ['conll2003']

ue_configs = {
    'calibrate': ['False'],
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'subsample_perc_val': [0.1],
    'subsample_perc': [0.1],
    'sn_value': [0.4, 0.6, 0.8, 1, 2, 3],
    'use_spectralnorm_inference': ['False'],
    'use_spectralnorm_train': ['True'],
}

cuda_devices = '[0,1,2,3,4,5]'


generate_bash([ue_configs], cuda_devices, tasks, 
              [generate_train_str_config, generate_ue_eval_methods_str_config], filename='run_train_eval_det_models_with_new_sn_params_deberta.sh')

## Deberta optimal MDSN 

In [52]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('+ue.sn_value\={}'.format(config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    config_list.append('training\=electra_base')
    return config_list

deberta_sn_values = {
    'cola': {
        'metric': 0.8,
        'reg': 0.4,
        'raw': 0.4,
    },
    'mrpc': {
        'metric': 1,
        'reg': 0.6,
        'raw': 0.8,
    },
    'sst2': {
        'metric': 0.6,
        'reg': 0.6,
        'raw': 0.6,
    },
    'conll2003': {
        'metric': 2,
        'reg': 1,
        'raw': 1,
    },
}

In [53]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                conf['sn_value'] = deberta_sn_values[task_name][reg]
                args = ' '.join(generate_func[i](conf))
                
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ue_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [54]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_deberta_models_glue_sn.sh')

In [55]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('training.per_device_eval_batch_size\=64') #due to out-of-memory

    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_model'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/0.0/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [56]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mahalanobis'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False'],
    'use_spectralnorm_model': ['True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_deberta_models_glue_det_best_sn.sh')

In [57]:
full_test = 'bash run_train_deberta_models_glue_sn.sh\n'
full_test += 'bash run_eval_deberta_models_glue_det_best_sn.sh'

with open (f'../../scripts/ue_scripts/run_deberta_new_sn_params.sh', 'w') as rsh:
    rsh.write(full_test)

In [58]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('+ue.sn_value\={}'.format(config['sn_value']))
    
    config_list.append("model.model_name_or_path\=microsoft/deberta-base")
    config_list.append('training\=electra_base')
    return config_list

In [59]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                conf['sn_value'] = deberta_sn_values[task_name][reg]
                
                new_task += f' script=run_conll2003.py'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ue_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [60]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_deberta_models_sn.sh')

In [61]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc_val\={}'.format(config['validation_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_model'] == 'True' else 'no_sn'
                
                with open(f'../../optimal_hyperparameters/deberta_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ue.margin\={margin}'
                args += f' ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/deberta_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/deberta_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [62]:
tasks = ['conll2003']

ue_configs = {
    'ue': ['mahalanobis'],
    'validation_subsample': [0.1],
    'use_spectralnorm': ['False'],
    'use_spectralnorm_model': ['True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_deberta_models_det_best_sn.sh')

In [63]:
full_test = 'bash run_train_deberta_models_sn.sh\n'
full_test += 'bash run_eval_deberta_models_det_best_sn.sh'

with open (f'../../scripts/ue_scripts/run_deberta_ner_new_sn_params.sh', 'w') as rsh:
    rsh.write(full_test)

# SNGP NER

In [ ]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    
    #config_list.append('training.per_device_eval_batch_size\={}'.format(dpp_config['per_device_eval_batch_size']))
    #config_list.append('training.num_train_epochs\={}'.format(dpp_config['num_train_epochs']))
    #config_list.append('training.learning_rate\={}'.format(dpp_config['learning_rate']))

    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [ ]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_ner.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                subsample_perc = conf['subsample_perc']
                conf['num_train_epochs'] = 80 if  subsample_perc == 0.1 else 8
                args = ' '.join(generate_func[i](conf))
                with open(f'../workdir/sngp/{task_name}/sngp_/0.1/2021-10-11/09-46-55-0/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)
                
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-sngp/{task_name}/{subsample_perc}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-sngp/{task_name}/{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ner_scripts/run_train_models_sngp_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[1,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Eval SNGP NER

In [ ]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_model_series.yaml python run_conll2003_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                subsample_perc = conf['subsample_perc']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_ner_for_model_series/electra-reg-sngp/{task_name}/{subsample_perc}'
                else:
                    base_path=f' output_dir=../workdir/run_ner_for_model_series/electra-raw-sngp/{task_name}/{subsample_perc}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'../workdir/run_train_models/electra-reg-sngp/{task_name}/{subsample_perc}/models/{task_name}'
                else:
                    model_series_dir = f'../workdir/run_train_models/electra-raw-sngp/{task_name}/{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['conll2003']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
    'per_device_eval_batch_size': [64],
    'committee_size': [10]
}
    
cuda_devices = '[1,2,3]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp_ner.sh')

# Generate train scripts for sngp

In [ ]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    config_list.append('ue.sngp.ridge_factor\={}'.format(dpp_config['ridge_factor']))
    config_list.append('ue.sngp.momentum\={}'.format(dpp_config['momentum']))
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [ ]:
import yaml

def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                size = 0.0 if task_name!='sst2' else 0.1
                with open(f'../workdir/sngp/{task_name}/sngp_/{size}/2021-10-11/07-01-17-0/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)
                
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-sngp-correct-hp/{task_name}/{validation_subsample}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-sngp-correct-hp/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/miscl_scripts/run_train_models_sngp.sh', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
    'ridge_factor': [1e-4],
    'momentum': [0.999],
}
    
cuda_devices = '[0,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# SNGP HP search

In [ ]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    
    config_list.append('ue.sngp.ridge_factor\={}'.format(dpp_config['ridge_factor']))
    config_list.append('ue.sngp.momentum\={}'.format(dpp_config['momentum']))
    
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [ ]:
import yaml

def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                size = 0.0 if task_name!='sst2' else 0.1
                with open(f'../workdir/sngp/{task_name}/sngp_/{size}/2021-10-11/07-01-17-0/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)
                
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                
                ridge_factor = conf['ridge_factor']
                momentum = conf['momentum']
                
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/miscl_scripts/run_train_models_sngp_hp_search.sh', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
    'ridge_factor': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
    'momentum': [0.999, 0.99, 0.9],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[1,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

In [ ]:
def generate_eval_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    
    config_list.append('ue.sngp.ridge_factor\={}'.format(dpp_config['ridge_factor']))
    config_list.append('ue.sngp.momentum\={}'.format(dpp_config['momentum']))
    
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                ridge_factor = conf['ridge_factor']
                momentum = conf['momentum']
                
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'../workdir/run_train_models/electra-reg-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}/models/{task_name}'
                else:
                    model_series_dir = f'../workdir/run_train_models/electra-raw-sngp/{task_name}/{ridge_factor}_{momentum}_{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['mrpc', 'cola', 'sst2']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'committee_size': [10],
    'ridge_factor': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
    'momentum': [0.999, 0.99, 0.9],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[1,2,3]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_str_config], file_name='run_eval_sngp_hp.sh')

# Generate eval script for sngp

In [ ]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list

def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg-sngp-correct-hp/{task_name}/{validation_subsample}'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw-sngp-correct-hp/{task_name}/{validation_subsample}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'../workdir/run_train_models/electra-reg-sngp-correct-hp/{task_name}/{validation_subsample}/models/{task_name}'
                else:
                    model_series_dir = f'../workdir/run_train_models/electra-raw-sngp-correct-hp/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'committee_size': [10]
}
    
cuda_devices = '[1,2,3]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp.sh')

# Generate train script for all models

In [3]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [4]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{use_spectralnorm}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['reg-curr', 'raw'],#['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[0,1]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_electra.sh')

# Generate train script for NER

In [8]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [9]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_ner.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf)) + ' data.subsample_perc\=0.1 ue.calibrate\=True'
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../run_train_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [11]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['True'],
    'use_selective': ['True'],
    'reg_type': ['reg-curr', 'raw'],#['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.1]
}
    
cuda_devices = '[0,1,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Generate eval script for DPP and MC-Dropout

In [ ]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg/{task_name}/{validation_subsample}'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                max_frac = conf['max_frac']
                base_path+= f'/raw_dpp/{max_frac}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg/{task_name}/{validation_subsample}/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw/{task_name}/{validation_subsample}/'
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.3, 0.4, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'use_selective': ['True', 'False'],
    'use_ood_sampling': ['False'],
    'dropout_committee_size': [20],
    'validation_subsample': [0.1, 0.0],
    'is_reused_mask': ['False'],
}
    
cuda_devices = '[2]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_raw_dpp_models.sh')

In [ ]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw/{task_name}/0.0'
                
                dropout_subs = conf['dropout_subs']
                base_path+= f'/mc/{dropout_subs}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/0.0'

                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['last', 'all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'use_selective': ['True', 'False'],
    'validation_subsample': [0.0],
}

cuda_devices = '[3]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_mc_models.sh')

# Generate eval script for determenistic methods

In [ ]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\=False')

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    
    return config_list

In [ ]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_glue_for_model_series_sn/electra_{reg}_{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models_sn/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['nuq'],#['mahalanobis', 'mc_mahalanobis'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
}

cuda_devices = '[0,1]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_nuq.sh')

In [ ]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm'] if conf['use_spectralnorm']=='True' else ''
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
}

cuda_devices = '[3]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models.sh')

# Generate eval script for det. methods for NER

In [ ]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))
    config_list.append('ue.use_selective\={}'.format(mc_config['use_selective']))
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=False')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    
    return config_list

In [ ]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_model_series.yaml python run_conll2003_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.1'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.1'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.1/models/{task_name}'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.1/models/{task_name}'
            
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['conll2003']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.1],
    'use_spectralnorm': ['False', 'True'],
}

cuda_devices = '[0,1,2,3]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models_ner.sh')

# Generate script for det. methods (spectral norm)

In [ ]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm'] if conf['use_spectralnorm']=='True' else ''
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['True'],
}

cuda_devices = '[2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models_sn.sh')

# Generate scripts for train Deep Ensemble

In [ ]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        subsample_perc = 0.63 if task_name!='sst2' else 0.1
        args = f'data.subsample_perc\={subsample_perc} ue.calibrate\=True data.validation_subsample\=0.1'
        
        with open(f'../workdir/optimal_hyperparams/{task_name}_electra_raw_no_sn_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/miscl_scripts/run_train_ensemble_models.sh', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '[1,2,3]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble

In [ ]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_ensemble_series.yaml python run_glue_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'

        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir}'

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series.sh')

# Generate scripts for train Deep Ensemble NER

In [ ]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series_ner.yaml python ./run_train_ensemble_series_ner.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        args = 'data.subsample_perc\=0.1 ue.calibrate\=True data.subsample_perc_val\=0.1 ue.use_cache\=True ue.use_selective\=False'
        
        with open(f'../workdir/optimal_hyperparams/{task_name}_electra_raw_no_sn_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ner_scripts/run_train_ensemble_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['conll2003']

cuda_devices = '[1,2,3]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble NER

In [ ]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_ensemble_series.yaml python run_conll2003_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}_selective.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        
        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir} '

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
import os

tasks = ['conll2003']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series_ner.sh')

In [ ]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_ensemble_series.yaml python run_conll2003_for_mahalanobis_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        
        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir} '
        
        new_task+=f' output_dir=../workdir/run_conll2003_for_mahalanobis_ensemble_series'

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
import os

tasks = ['conll2003']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_mahalanobis_ensemble_series_ner.sh')

# Newsgroups ALL

In [ ]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list


def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_20newsgroups.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_newsgroups.sh')

# DPP HP

In [ ]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_20newsgroups_dpp_hp.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models_dpp_hp/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[2,7]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_dpp_hp.sh')

In [ ]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))

    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_raw_no_sn/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                max_frac = conf['max_frac']
                dropout_committee_size = conf['dropout_committee_size']
                base_path+= f'/ddpp_{dpp_type}_{max_frac}_{dropout_committee_size}/'
                new_task+=base_path
                
                model_series_dir = f'/notebook/uncertainty-estimation/workdir/run_train_models_dpp_hp/electra_raw_no_sn/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.3, 0.4, 0.5, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[2,7]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_dpp_models_hp_search_20ng.sh')

# Full

In [ ]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    config_list.append('training.per_device_eval_batch_size\=16')
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'True' if mc_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('training.per_device_eval_batch_size\=16')
    #config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                max_frac = conf['max_frac']
                dropout_committee_size = conf['dropout_committee_size']
                base_path+= f'/ddpp_{dpp_type}_best/'#_{max_frac}_{dropout_committee_size}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [ ]:
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.6], # best for both ddpp
    "committee_size": [100],
    'calibrate': ['True'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[4,5,6]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_20ng_dpp_models.sh')

In [ ]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                base_path+= f'/mc_all/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc'],
    "committee_size": [20],
    'calibrate': ['False'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'validation_subsample': [0.0],
    'dropout_subs': ['all']
}   
    
cuda_devices = '[4,5,6]'

generate_bash_mc([dpp_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_20ng_mc_models.sh')

In [ ]:
import os

def generate_bash_det(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                ue = conf['ue']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                base_path+= f'/{ue}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

        
def generate_eval_ue_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'True' if mc_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    config_list.append('ue.use_spectralnorm\=False')#.format(mc_config['use_spectralnorm']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    config_list.append('training.per_device_eval_batch_size\=16')

    return config_list

tasks = ['20newsgroups']

ue_configs = {
    'ue': ['mahalanobis'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
}

cuda_devices = '[4,5,6]'

generate_bash_det([ue_configs], cuda_devices, tasks, [generate_eval_ue_str_config], file_name='run_eval_20ng_det_models.sh')